In [1]:
import numpy as np
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt

In [2]:
from utils.get_svhn import *

In [3]:
# extract data into h5py format

trainData = h5py.File('data/train/digitStruct.mat', 'r')
testData = h5py.File('data/test/digitStruct.mat', 'r')

In [4]:
from utils.data_extractor import generate_trainable_data,preprocess_data

In [5]:
# define the size of the data after extracting
size = 64

# transform data into array

X_test, y_test = generate_trainable_data(testData, size, 'test')

C:\Users\zjyng\Anaconda3\envs\envTF113\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [6]:
# one hot encoding

y_test_dummy = tf.keras.utils.to_categorical(y_test)

# transform format of labels to fit the model

y_test_multi = []
for i in range(6):
    y_test_multi.append(y_test_dummy[:,i])

In [7]:
model = tf.keras.models.load_model('models/dense_extra')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [8]:
result = model.evaluate(X_test, y_test_multi)

13068/13068 [==============================] - 313s 24ms/sample - loss: 0.7718 - dense_10_loss: 0.1230 - dense_11_loss: 0.2384 - dense_12_loss: 0.2808 - dense_13_loss: 0.1132 - dense_14_loss: 0.0143 - dense_15_loss: 0.0020 - dense_10_acc: 0.9797 - dense_11_acc: 0.9492 - dense_12_acc: 0.9450 - dense_13_acc: 0.9795 - dense_14_acc: 0.9974 - dense_15_acc: 0.9998


In [9]:
result[7:]

[0.9797215, 0.9491889, 0.9449801, 0.9794919, 0.9973982, 0.99984694]

In [10]:
y_pred = np.array(model.predict(X_test))

In [43]:
pred_label = np.transpose(y_pred.argmax(axis=2))

In [44]:
# compare predictions and gound truth

import pandas as pd

pd_test = pd.DataFrame(y_test)
pd_pred = pd.DataFrame(pred_label)

com_result = pd_test == pd_pred

com_result

,0,1,2,3,4,5
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,True,True,True,True
3,True,True,True,True,True,True
4,True,True,True,True,True,True
...,...,...,...,...,...,...
13063,True,True,True,True,True,True
13064,True,True,True,True,True,True
13065,True,True,True,True,True,True
13066,True,True,True,True,True,True


In [45]:
# extract the correct predictions to calculate the accuracy

num = []

for i in range(len(com_result)):
    if all(com_result.iloc[i]):
        num.append(i)

correct_result = com_result.iloc[num]
correct_result

,0,1,2,3,4,5
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,True,True,True,True
3,True,True,True,True,True,True
4,True,True,True,True,True,True
...,...,...,...,...,...,...
13063,True,True,True,True,True,True
13064,True,True,True,True,True,True
13065,True,True,True,True,True,True
13066,True,True,True,True,True,True


In [46]:
# overall accuracy

print("acc: {}".format(len(correct_result)/pred_label.shape[0]))

acc: 0.8974594429139884


In [50]:
# extract wrong predictions

false_pred = com_result.drop(correct_result.index)
false_pred

,0,1,2,3,4,5
12,True,True,False,True,True,True
16,True,True,False,True,True,True
20,True,False,False,False,True,True
29,True,False,False,True,True,True
30,True,True,False,True,True,True
...,...,...,...,...,...,...
12983,False,True,False,False,True,True
13000,True,True,False,True,True,True
13015,True,True,False,True,True,True
13020,True,True,False,True,True,True


In [53]:
# write wrong predictions together with the ground truth into a csv file to see the difference

import csv
with open('result_compare.csv','w') as csvfile:
    fieldnames = ['Id','pred_label','true_label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in false_pred.index:
        filename = str(i+1) + '.png'
        Pred_label = str(pred_label[i])
        True_label = str(y_test[i])
        writer.writerow({'Id': filename, 'pred_label': Pred_label, 'true_label': True_label})

In [124]:
from PIL import Image

n = 561

img = Image.fromarray(np.uint8(X_test[n]))
img.show()

print("prediction: {}".format(pred_label[n]), "ground truth: {}".format(y_test[n]), sep='\n')

prediction: [2 8 8 0 0 0]
ground truth: [2. 8. 3. 0. 0. 0.]
